In [36]:
!pip install h5py
!pip install typing-extensions
!pip install wheel

In [37]:
!pip install peft==0.10.0

In [38]:
!pip install transformers datasets scikit-learn torch --quiet --use-deprecated=legacy-resolver

ERROR: pip's legacy dependency resolver does not consider dependency conflicts when selecting packages. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you'll have fsspec 2025.3.0 which is incompatible.


In [39]:
!pip install fsspec==2025.3.2

  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.3.2-py3-none-any.whl (194 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.


In [40]:
!pip install adapters

In [41]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from datasets import Dataset

In [42]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [43]:
from transformers import TrainingArguments, Trainer

In [44]:
from transformers import TrainingArguments, EvalPrediction
from  adapters import AdapterTrainer

In [45]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns

In [46]:
# === 1. ЗАГРУЗКА ДАННЫХ ===

# Загрузка предварительно подготовленного датафрейма с колонками "clean_title" и "target"
df = pd.read_csv("df_news_clean.csv")  # датасет с курсами и новостями, сохраненный из предыдущего эксперимента

In [47]:
#df = df.dropna(subset=["clean_title", "target"])
df_text = df[["title", "target"]].rename(columns={"title": "text", "target": "label"}).dropna()

# Приводим метки к типу int
df_text["label"] = df_text["label"].astype(int)
df_text

,text,label
0,Глава Генштаба оценил вероятность начала крупн...,0
1,Путин назвал подвигом жизнь россиян в 1990-х П...,0
2,Генсек ООН заявил о максимальном уровне геопол...,0
3,В США объяснили путаницу с выводом войск из Ир...,0
4,Путин рассказал о попытках помешать отношениям...,0
...,...,...
798,Церемония встречи Путина и Ким Чен Ына началас...,1
799,Генсек НАТО отреагировал на договор России и С...,1
800,Бастрыкин рассказал про планы нацистов истреби...,0
801,Пентагон спрогнозировал напряженность отношени...,0


In [48]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruBert-base")

from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("sberbank-ai/ruBert-base", num_labels=2)

from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
from datasets import Dataset

# Преобразуем датафрейм в Dataset
raw_ds = Dataset.from_pandas(df_text)

def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

# Токенизация
tokenized_ds = raw_ds.map(tokenize_function, batched=True)
tokenized_ds = tokenized_ds.rename_column("label", "labels")
tokenized_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Делим на train/test
split = len(tokenized_ds) - 200
train_ds = tokenized_ds.select(range(split))
test_ds = tokenized_ds.select(range(split, len(tokenized_ds)))


Map:   0%|          | 0/803 [00:00<?, ? examples/s]

In [50]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    report_to="none",
)

In [51]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.793700,0.686549,0.560000,0.560000,1.000000,0.717949
2,0.692600,0.708373,0.470000,0.625000,0.133929,0.220588
3,0.705100,0.692912,0.470000,0.625000,0.133929,0.220588


TrainOutput(global_step=228, training_loss=0.7088936776445623, metrics={'train_runtime': 3183.9734, 'train_samples_per_second': 0.568, 'train_steps_per_second': 0.072, 'total_flos': 118991974786560.0, 'train_loss': 0.7088936776445623, 'epoch': 3.0})

In [52]:
# === 8. ОЦЕНКА ===

preds = trainer.predict(test_ds)
y_true = preds.label_ids
y_pred = np.argmax(preds.predictions, axis=1)

print("\n📋 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))


📋 Classification Report:

              precision    recall  f1-score   support

           0      0.449     0.898     0.598        88
           1      0.625     0.134     0.221       112

    accuracy                          0.470       200
   macro avg      0.537     0.516     0.410       200
weighted avg      0.547     0.470     0.387       200



In [ ]:
# === 9. CONFUSION MATRIX ===
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["0 (↓)", "1 (↑)"], yticklabels=["0 (↓)", "1 (↑)"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix (Test Set)")
plt.show()

In [54]:
# === 10. СОХРАНЕНИЕ МОДЕЛИ ===
trainer.save_model("./finetuned-rubert")
tokenizer.save_pretrained("./finetuned-rubert")

('./finetuned-rubert/tokenizer_config.json',
 './finetuned-rubert/special_tokens_map.json',
 './finetuned-rubert/vocab.txt',
 './finetuned-rubert/added_tokens.json',
 './finetuned-rubert/tokenizer.json')